# Data Clustering & Preprocessing Pipeline
Split cleaned real estate data into rent/sale, normalize prices, cluster by price segment (low/medium/high), and preprocess for modeling.

## 1. Load Libraries and Dataset

In [7]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load the fixed CSV
data_path = Path.cwd().parent / "data" / "processed" / "cleaned_real_estate_fixed.csv"
if not data_path.exists():
    data_path = Path.home() / "Desktop" / "ML-project" / "ML" / "data" / "processed" / "cleaned_real_estate_fixed.csv"

df = pd.read_csv(data_path)

print(f"Dataset loaded: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")

Dataset loaded: (9582, 8)
Columns: ['price', 'transaction', 'city', 'region', 'surface', 'bathrooms', 'rooms', 'property_type']

First few rows:
      price transaction     city               region  surface  bathrooms  \
0    2100.0        rent    Tunis  Jardins De Carthage    120.0          2   
1     850.0        rent   Sousse       Sousse Jawhara     90.0          1   
2  500000.0        sale   Ariana            La Soukra    144.0          2   
3     850.0        rent  Bizerte            Ras Jebel    400.0          1   
4    1800.0        rent    Tunis    Ain Zaghouan Nord    130.0          1   

   rooms property_type  
0      2   Appartement  
1      2   Appartement  
2      3   Appartement  
3      2         Villa  
4      3         Villa  

Data types:
price            float64
transaction       object
city              object
region            object
surface          float64
bathrooms          int64
rooms              int64
property_type     object
dtype: object

Missing values

## 2. Split Dataset into Rent and Sale

In [8]:
# Identify the transaction type column
print("Unique values in potential type columns:")
for col in df.columns:
    if 'type' in col.lower() or 'transaction' in col.lower() or 'offer' in col.lower():
        print(f"  {col}: {df[col].unique()}")

# Split based on transaction type (prioritize 'transaction' column)
type_col = None
for col in df.columns:
    if col.lower() == 'transaction':
        type_col = col
        break

if type_col is None:
    for col in df.columns:
        if 'type' in col.lower():
            type_col = col
            break

print(f"\nUsing column for split: '{type_col}'")

if type_col:
    print(f"Values in {type_col}: {df[type_col].unique()}")
    rent_df = df[df[type_col].str.lower() == 'rent'].copy()
    sale_df = df[df[type_col].str.lower() == 'sale'].copy()
else:
    print("Error: Could not find transaction type column!")
    rent_df = pd.DataFrame()
    sale_df = pd.DataFrame()

print(f"\nRent listings: {len(rent_df):,}")
print(f"Sale listings: {len(sale_df):,}")
print(f"Total: {len(rent_df) + len(sale_df):,}")

if len(rent_df) > 0:
    print(f"\nRent price stats:")
    print(rent_df['price'].describe())

if len(sale_df) > 0:
    print(f"\nSale price stats:")
    print(sale_df['price'].describe())

Unique values in potential type columns:
  transaction: ['rent' 'sale']
  property_type: ['Appartement' 'Villa' 'Maison' 'Studio']

Using column for split: 'transaction'
Values in transaction: ['rent' 'sale']

Rent listings: 5,570
Sale listings: 4,012
Total: 9,582

Rent price stats:
count     5570.000000
mean      1473.180610
std       1397.906858
min        200.000000
25%        850.000000
50%       1100.000000
75%       1457.500000
max      15000.000000
Name: price, dtype: float64

Sale price stats:
count    4.012000e+03
mean     4.797959e+05
std      4.005123e+05
min      2.000000e+04
25%      2.500000e+05
50%      3.500000e+05
75%      5.900000e+05
max      3.500000e+06
Name: price, dtype: float64


## 3. Preprocess Features (Encoding + Cleaning)

In [9]:
def preprocess_data(data, dataset_name=""):
    """Preprocess: encode categoricals, select features (no row dropping)"""
    df_proc = data.copy()
    
    # We assume no missing values; keep all rows
    print(f"{dataset_name} - Rows: {len(df_proc):,}")
    
    # Select numeric and categorical features
    numeric_cols = ['price', 'surface', 'rooms', 'bedrooms', 'bathrooms']
    numeric_cols = [c for c in numeric_cols if c in df_proc.columns]
    
    categorical_cols = ['city', 'region']
    categorical_cols = [c for c in categorical_cols if c in df_proc.columns]
    
    # Fill missing numeric values with median (if any)
    for col in numeric_cols:
        if df_proc[col].isnull().sum() > 0:
            df_proc[col].fillna(df_proc[col].median(), inplace=True)
    
    # Encode categorical features using LabelEncoder
    le_dict = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df_proc[f'{col}_encoded'] = le.fit_transform(df_proc[col].astype(str))
        le_dict[col] = le
    
    print(f"{dataset_name} - Feature columns: {numeric_cols}")
    print(f"{dataset_name} - Categorical encoded: {[f'{c}_encoded' for c in categorical_cols]}")
    
    return df_proc, numeric_cols, categorical_cols, le_dict

# Preprocess both datasets
rent_processed, rent_numeric, rent_cat, rent_le = preprocess_data(rent_df, "RENT")
sale_processed, sale_numeric, sale_cat, sale_le = preprocess_data(sale_df, "SALE")

print(f"\nAfter preprocessing:")
print(f"  Rent: {len(rent_processed):,} rows")
print(f"  Sale: {len(sale_processed):,} rows")

RENT - Rows: 5,570
RENT - Feature columns: ['price', 'surface', 'rooms', 'bathrooms']
RENT - Categorical encoded: ['city_encoded', 'region_encoded']
SALE - Rows: 4,012
SALE - Feature columns: ['price', 'surface', 'rooms', 'bathrooms']
SALE - Categorical encoded: ['city_encoded', 'region_encoded']

After preprocessing:
  Rent: 5,570 rows
  Sale: 4,012 rows


## 4. Normalize Price Column

In [10]:
# Normalize prices separately for rent and sale using StandardScaler
rent_scaler = StandardScaler()
sale_scaler = StandardScaler()

# Apply scaling
rent_processed['price_normalized'] = rent_scaler.fit_transform(rent_processed[['price']])
sale_processed['price_normalized'] = sale_scaler.fit_transform(sale_processed[['price']])

print("Price Normalization (StandardScaler):")
print(f"RENT - Original range: [{rent_processed['price'].min():.0f}, {rent_processed['price'].max():.0f}]")
print(f"RENT - Normalized range: [{rent_processed['price_normalized'].min():.3f}, {rent_processed['price_normalized'].max():.3f}]")
print(f"\nSALE - Original range: [{sale_processed['price'].min():.0f}, {sale_processed['price'].max():.0f}]")
print(f"SALE - Normalized range: [{sale_processed['price_normalized'].min():.3f}, {sale_processed['price_normalized'].max():.3f}]")

Price Normalization (StandardScaler):
RENT - Original range: [200, 15000]
RENT - Normalized range: [-0.911, 9.677]

SALE - Original range: [20000, 3500000]
SALE - Normalized range: [-1.148, 7.542]


## 5. Cluster Rent Listings by Price Segment

In [11]:
def apply_clustering(data, dataset_name="", n_clusters=3):
    """
    Apply KMeans clustering on ALL FEATURES (surface, rooms, bedrooms, bathrooms).
    Then analyze price segments by city/region to determine if a location is high/low/medium value.
    """
    df_clust = data.copy()
    
    # Prepare features for clustering: ALL numeric features EXCEPT price
    clustering_features = ['surface', 'rooms', 'bedrooms', 'bathrooms']
    clustering_features = [c for c in clustering_features if c in df_clust.columns]
    
    # Normalize clustering features
    scaler_features = StandardScaler()
    X_cluster_normalized = scaler_features.fit_transform(df_clust[clustering_features])
    
    # Apply KMeans on normalized features (NOT price)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df_clust['property_type_cluster'] = kmeans.fit_predict(X_cluster_normalized)
    
    # Analyze clusters: what do they represent in terms of property characteristics?
    print(f"\n{dataset_name} - Property Type Clusters (based on surface, rooms, bedrooms, bathrooms):")
    for cluster_id in sorted(df_clust['property_type_cluster'].unique()):
        cluster_data = df_clust[df_clust['property_type_cluster'] == cluster_id]
        print(f"\n  Cluster {cluster_id}:")
        print(f"    Count: {len(cluster_data):,}")
        print(f"    Avg Surface: {cluster_data['surface'].mean():.1f}")
        print(f"    Avg Rooms: {cluster_data['rooms'].mean():.1f}")
        print(f"    Avg Price: {cluster_data['price'].mean():,.0f}")
        print(f"    Price Range: [{cluster_data['price'].min():,.0f}, {cluster_data['price'].max():,.0f}]")
    
    # Now analyze price segments BY CITY/REGION to determine if that location is high/low/medium
    print(f"\n{dataset_name} - Analyzing price value by City/Region...")
    
    # Calculate city/region market statistics
    city_region_stats = df_clust.groupby(['city', 'region']).agg({
        'price': ['mean', 'median', 'count'],
        'surface': 'mean',
        'rooms': 'mean'
    }).reset_index()
    city_region_stats.columns = ['city', 'region', 'mean_price', 'median_price', 'count', 'avg_surface', 'avg_rooms']
    
    # Sort by mean price to determine thresholds
    all_mean_prices = df_clust.groupby(['city', 'region'])['price'].mean()
    price_threshold_low = all_mean_prices.quantile(0.33)
    price_threshold_high = all_mean_prices.quantile(0.67)
    
    # Assign price segment based on city/region's mean price
    def assign_segment(mean_price):
        if mean_price < price_threshold_low:
            return 'Low'
        elif mean_price < price_threshold_high:
            return 'Medium'
        else:
            return 'High'
    
    city_region_stats['price_segment'] = city_region_stats['mean_price'].apply(assign_segment)
    
    # Merge segment back to main dataframe
    df_clust = df_clust.merge(city_region_stats[['city', 'region', 'price_segment']], 
                               on=['city', 'region'], how='left')
    
    print(f"\n{dataset_name} - Price Thresholds (by City/Region mean price):")
    print(f"  Low:    < {price_threshold_low:,.0f}")
    print(f"  Medium: {price_threshold_low:,.0f} - {price_threshold_high:,.0f}")
    print(f"  High:   > {price_threshold_high:,.0f}")
    
    return df_clust, kmeans, scaler_features, city_region_stats

# Apply clustering to RENT
rent_clustered, rent_kmeans, rent_scaler, rent_stats = apply_clustering(rent_processed, "RENT", n_clusters=3)
print("\n" + "="*80)

# Apply clustering to SALE
sale_clustered, sale_kmeans, sale_scaler, sale_stats = apply_clustering(sale_processed, "SALE", n_clusters=3)


RENT - Property Type Clusters (based on surface, rooms, bedrooms, bathrooms):

  Cluster 0:
    Count: 4,068
    Avg Surface: 94.8
    Avg Rooms: 1.8
    Avg Price: 1,084
    Price Range: [200, 12,345]

  Cluster 1:
    Count: 146
    Avg Surface: 927.0
    Avg Rooms: 7.1
    Avg Price: 4,966
    Price Range: [350, 15,000]

  Cluster 2:
    Count: 1,356
    Avg Surface: 244.6
    Avg Rooms: 3.6
    Avg Price: 2,266
    Price Range: [200, 14,000]

RENT - Analyzing price value by City/Region...

RENT - Price Thresholds (by City/Region mean price):
  Low:    < 804
  Medium: 804 - 1,192
  High:   > 1,192


SALE - Property Type Clusters (based on surface, rooms, bedrooms, bathrooms):

  Cluster 0:
    Count: 1,156
    Avg Surface: 381.6
    Avg Rooms: 4.1
    Avg Price: 663,622
    Price Range: [38,000, 3,500,000]

  Cluster 1:
    Count: 2,534
    Avg Surface: 124.9
    Avg Rooms: 2.2
    Avg Price: 311,194
    Price Range: [30,000, 3,399,996]

  Cluster 2:
    Count: 322
    Avg Surface:

## 6. Summary: Cluster Distribution by City/Region

In [12]:
# Summary of cluster distribution by city
print("=" * 80)
print("RENT - Cluster distribution by City:")
print("=" * 80)
rent_summary = rent_clustered.groupby(['city', 'price_segment']).size().unstack(fill_value=0)
print(rent_summary)

print("\n" + "=" * 80)
print("SALE - Cluster distribution by City:")
print("=" * 80)
sale_summary = sale_clustered.groupby(['city', 'price_segment']).size().unstack(fill_value=0)
print(sale_summary)

# Sample of clustered data
print("\n" + "=" * 80)
print("Sample of RENT clustered data:")
print("=" * 80)
print(rent_clustered[['city', 'region', 'price', 'price_normalized', 'price_segment']].head(10))

print("\n" + "=" * 80)
print("Sample of SALE clustered data:")
print("=" * 80)
print(sale_clustered[['city', 'region', 'price', 'price_normalized', 'price_segment']].head(10))

RENT - Cluster distribution by City:
price_segment  High  Low  Medium
city                            
Ariana          595   68     295
Ben Arous         0  141     214
Bizerte           0    8     115
Béja              2    4       0
Gabès             0   13       0
Jendouba          0    3      22
Kairouan          0    5       0
Kasserine         0    1       0
Kébili            0    2       0
La Manouba        0   55       3
Mahdia            0    0     174
Monastir          6    5     133
Médenine         12    0      30
Nabeul          493    6     167
Sfax             17   61      46
Sousse           15   36     517
Tataouine         0    0       1
Tozeur            0    4       0
Tunis          1666   63     571
Zaghouan          0    1       0

SALE - Cluster distribution by City:
price_segment  High  Low  Medium
city                            
Ariana          344   37     355
Ben Arous        65  207     161
Bizerte           0   48      25
Béja              0    1       3
G

In [14]:
# Build lookup table from training data
rent_lookup = rent_clustered[['city', 'region', 'price_segment']].drop_duplicates()
sale_lookup = sale_clustered[['city', 'region', 'price_segment']].drop_duplicates()

# Save lookup tables for inference
output_dir = Path.cwd().parent / "data" / "processed"
if not output_dir.exists():
    output_dir = Path.home() / "Desktop" / "ML-project" / "ML" / "data" / "processed"

rent_lookup_path = output_dir / "rent_city_region_segment.csv"
sale_lookup_path = output_dir / "sale_city_region_segment.csv"

rent_lookup.to_csv(rent_lookup_path, index=False)
sale_lookup.to_csv(sale_lookup_path, index=False)

print(f"✅ Rent city/region segment lookup saved to: {rent_lookup_path}")
print(f"✅ Sale city/region segment lookup saved to: {sale_lookup_path}")

# Example: apply lookup to new data (must contain city & region)
# new_input_df = pd.DataFrame([{"city": "Tunis", "region": "Centre Ville"}])
# new_input_df = new_input_df.merge(rent_lookup, on=['city', 'region'], how='left')
# print(new_input_df)


✅ Rent city/region segment lookup saved to: C:\Users\Mediatek\Desktop\ML-project\ML\data\processed\rent_city_region_segment.csv
✅ Sale city/region segment lookup saved to: C:\Users\Mediatek\Desktop\ML-project\ML\data\processed\sale_city_region_segment.csv


In [15]:
# Save clustered datasets
output_dir = Path.cwd().parent / "data" / "processed"
if not output_dir.exists():
    output_dir = Path.home() / "Desktop" / "ML-project" / "ML" / "data" / "processed"

rent_output_path = output_dir / "rent_clustered.csv"
sale_output_path = output_dir / "sale_clustered.csv"

# Save, keeping original columns + new ones (only if present)
base_cols = ['city', 'region', 'price', 'price_normalized', 'surface', 'rooms', 'bedrooms', 'bathrooms', 'property_type', 'property_type_cluster', 'price_segment']

rent_cols = [c for c in base_cols if c in rent_clustered.columns]
sale_cols = [c for c in base_cols if c in sale_clustered.columns]

rent_save = rent_clustered[rent_cols].copy()
sale_save = sale_clustered[sale_cols].copy()

rent_save.to_csv(rent_output_path, index=False)
sale_save.to_csv(sale_output_path, index=False)

print(f"✅ Rent dataset saved to: {rent_output_path}")
print(f"   - Shape: {rent_save.shape}")
print(f"   - Columns: {list(rent_save.columns)}")

print(f"\n✅ Sale dataset saved to: {sale_output_path}")
print(f"   - Shape: {sale_save.shape}")
print(f"   - Columns: {list(sale_save.columns)}")

# Also save summary statistics
summary_stats = pd.DataFrame({
    'Dataset': ['RENT', 'SALE'],
    'Total_Rows': [len(rent_clustered), len(sale_clustered)],
    'Low_Segment': [
        len(rent_clustered[rent_clustered['price_segment'] == 'Low']),
        len(sale_clustered[sale_clustered['price_segment'] == 'Low'])
    ],
    'Medium_Segment': [
        len(rent_clustered[rent_clustered['price_segment'] == 'Medium']),
        len(sale_clustered[sale_clustered['price_segment'] == 'Medium'])
    ],
    'High_Segment': [
        len(rent_clustered[rent_clustered['price_segment'] == 'High']),
        len(sale_clustered[sale_clustered['price_segment'] == 'High'])
    ],
    'Avg_Price': [
        rent_clustered['price'].mean(),
        sale_clustered['price'].mean()
    ]
})

summary_path = output_dir / "clustering_summary.csv"
summary_stats.to_csv(summary_path, index=False)

print(f"\n✅ Summary statistics saved to: {summary_path}")
print("\nSummary Statistics:")
print(summary_stats.to_string(index=False))

✅ Rent dataset saved to: C:\Users\Mediatek\Desktop\ML-project\ML\data\processed\rent_clustered.csv
   - Shape: (5570, 10)
   - Columns: ['city', 'region', 'price', 'price_normalized', 'surface', 'rooms', 'bathrooms', 'property_type', 'property_type_cluster', 'price_segment']

✅ Sale dataset saved to: C:\Users\Mediatek\Desktop\ML-project\ML\data\processed\sale_clustered.csv
   - Shape: (4012, 10)
   - Columns: ['city', 'region', 'price', 'price_normalized', 'surface', 'rooms', 'bathrooms', 'property_type', 'property_type_cluster', 'price_segment']

✅ Summary statistics saved to: C:\Users\Mediatek\Desktop\ML-project\ML\data\processed\clustering_summary.csv

Summary Statistics:
Dataset  Total_Rows  Low_Segment  Medium_Segment  High_Segment     Avg_Price
   RENT        5570          476            2288          2806   1473.180610
   SALE        4012          931            1778          1303 479795.927966
